In [2]:
# from fastai import learner
from fastai.vision.all import *
import numpy as np
import os
import pandas as pd
import librosa as lb
import librosa.display
import soundfile as sf
import matplotlib.patches as patch
import matplotlib.pyplot as plt
import IPython.display as ipd
from pathlib import Path
from tqdm.notebook import tqdm
from scipy.io import wavfile
from torch import cuda
import shutil

CC_ROOT=Path("D:\\KidElectric\\rfcx_kaggle") #Cookiecutter datascience-style project
CC_DATA_TEST=CC_ROOT.joinpath('data').joinpath('interim').joinpath('test')
CC_MODEL_ROOT=CC_ROOT.joinpath('models').joinpath('fit')

DATA_ROOT = CC_ROOT.joinpath('data')
TRAIN_AUDIO_ROOT = DATA_ROOT.joinpath('raw').joinpath('train')#Update to point to cookiecutter data/raw/train
TEST_AUDIO_ROOT = DATA_ROOT.joinpath('raw').joinpath('test')#Update to point to cookiecutter data/raw/test


df_train = pd.DataFrame({
    "recording_id": [path.stem for path in TRAIN_AUDIO_ROOT.glob("*.flac")],
})

df_test = pd.DataFrame({
    "recording_id": [path.stem for path in TEST_AUDIO_ROOT.glob("*.flac")],
})


df_tp=pd.read_csv(CC_ROOT.joinpath('references').joinpath('train_tp.csv')).set_index('recording_id')

df_fp=pd.read_csv(CC_ROOT.joinpath('references').joinpath('train_fp.csv')).set_index('recording_id')


In [26]:
cuda.empty_cache()

In [112]:
#Load a model:
def which_singer(x): return x.parts[-1].split('_')[1] #Need to define this function first
# model_name='rn101_50ep_512_clip_full_aug2_noflip_FB_120320_mixup0p5.pkl'
model_name='rn101_50ep_lr8p32e-10_434_clip_TP_FP_NP_aug2_noflip_120420_mixup0p5.pkl'
learn=load_learner(CC_MODEL_ROOT.joinpath(model_name))

In [113]:
# Predict many images at once 
imgs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob('*.png')]
dl = learn.dls.test_dl(imgs, item_tfms=Resize(512,ResizeMethod.Squish),num_workers=0,bs=64)
a,b=learn.get_preds(dl=dl) # or torch.softmax?
prob_order= learn.dls.vocab
recs=df_test['recording_id']
df_prob=pd.DataFrame(index=df_test['recording_id'],columns=['s%d' % i for i in range(0,24,1)])
m = nn.Softmax(dim=1)
probs=m(a)
for rec in recs:
    temp=np.zeros((len(pngs),len(prob_order)))
    n=0
    for i,img in enumerate(imgs):
        if rec == img.parts[-1].split('_')[0]:
            temp[n,:]=probs[i,:]
            n += 1
    mp=np.mean(temp,axis=0) #Or should it be mean? -- doesn't seem to matter!
    for i,spec in enumerate(prob_order):
        if int(spec) < 24:
            df_prob['s%s' % spec][rec]=float(mp[i])
df_prob.to_csv(CC_MODEL_ROOT.parent.joinpath('predictions').joinpath('%s-sum.csv' % model_name[0:-4]))
print('Finished')

Finished


In [8]:
# Predict many images from many models:
rs = 33
nmodels=20
path =CC_ROOT.joinpath('data').joinpath('processed').joinpath('resamp_%d' % rs)
imgs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob('*.png')]
im_size=512 # 224
nspec=24
def which_singer(x): return x.parts[-1].split('_')[1] #Need to define this function first
for ii in range(nmodels):
#     model_name='rn50_50ep_%d_resamp%d_TP_FP_NP_aug2_noflip_%d.pkl' % (im_size,rs,ii) #Each of these models was pretty bad on their own (~0.65 error rate)
#     model_name='rn101_50ep_%d_resamp%d_TP_FP_NP_aug2_noflip_mixup0p5_%d.pkl' % (im_size,rs,ii)
    model_name='rn18_30ep_%d_resamp%d_TP_aug1_noflip_%d.pkl' %(im_size,rs,ii)
    mpath=path.joinpath('models')
    print('Loading model %s' % mpath.joinpath(model_name))
    learn=load_learner(mpath.joinpath(model_name))
    dl = learn.dls.test_dl(imgs, item_tfms=Resize(im_size,ResizeMethod.Squish),num_workers=0,bs=64)
    print('\tPredicting...')
    a,b=learn.get_preds(dl=dl) # or torch.softmax?
    prob_order= learn.dls.vocab
    recs=df_test['recording_id']
    df_prob=pd.DataFrame(index=df_test['recording_id'],columns=['s%d' % i for i in range(0,24,1)])
    m = nn.Softmax(dim=1)
    probs=m(a)
    for rec in recs:
        temp=np.zeros((len(imgs),len(prob_order)))
        n=0
        for i,img in enumerate(imgs):
            if rec == img.parts[-1].split('_')[0]:
                temp[n,:]=probs[i,:]
                n += 1
        mp=np.mean(temp,axis=0) #Or should it be mean? -- doesn't seem to matter!
        for i,spec in enumerate(prob_order):
            if int(spec) < nspec:
                df_prob['s%s' % spec][rec]=float(mp[i])
    print('\tSaving')
    df_prob.to_csv(mpath.joinpath('%s.csv' % model_name[0:-4]))
    
#Load in multiple .csv and take the mean:
csv_fns=mpath.glob( model_name[0:-7] + '*.csv')
df_load={}
for i,fn in enumerate(csv_fns):
    df_load[i]= pd.read_csv(fn)
b=df_load[0]
for col in df_load[0].columns:
    if col != 'recording_id':
        for k in df_load.keys():
            b[col]=b[col] + df_load[k][col]
        b[col]= b[col]/nspec
b.head()
b.to_csv(mpath.joinpath('%s_%d_averaged.csv' % (model_name[0:-4],len(df_load))), index=False)
print('Finished')

Loading model D:\KidElectric\rfcx_kaggle\data\processed\resamp_33\models\rn18_30ep_512_resamp33_TP_aug1_noflip_0.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\KidElectric\\rfcx_kaggle\\data\\processed\\resamp_33\\models\\rn18_30ep_512_resamp33_TP_aug1_noflip_0.pkl'

In [120]:
#Load in multiple .csv and take the mean:
csv_fns=mpath.glob( model_name[0:-7] + '*.csv')
df_load={}
for i,fn in enumerate(csv_fns):
    df_load[i]= pd.read_csv(fn)
b=df_load[0]
for col in df_load[0].columns:
    if col != 'recording_id':
        for k in df_load.keys():
            b[col]=b[col] + df_load[k][col]
        b[col]= b[col]/nspec
b.head()
b.to_csv(mpath.joinpath('%s_%d_averaged.csv' % (model_name[0:-4],len(df_load))), index=False)
print('Finished')

Finished


In [121]:
len(df_load)

20

In [122]:
mpath

Path('D:/KidElectric/rfcx_kaggle/data/processed/resamp_15/models')